# Trading @ Georgia Tech Exchange Client

## Constants

In [ ]:
# TODO: fill these constants
USERNAME = ""
API_KEY = ""

# do not modify
RATE_LIMIT = 15
URI = 'ec2-3-16-107-184.us-east-2.compute.amazonaws.com'
URL = f"http://{URI}:8080"
WS_URL = f"ws://{URI}:8080/exchange-socket"

## Importing Dependencies

In [ ]:

!pip install sortedcontainers aiohttp websockets pandas numpy


In [ ]:

from __future__ import annotations

import aiohttp
import asyncio
import contextlib
import copy
import json
import time
import traceback
import urllib.request
from abc import ABC
from abc import abstractmethod
from collections import deque
from dataclasses import dataclass
from enum import Enum
from typing import Any

import nest_asyncio
import websockets
from sortedcontainers import SortedDict

nest_asyncio.apply()

import sys
sys.path.append('./')
from client_utils import *


## Strategy Implementation

---

### Instructions

Implement your strategy here! There are 2 methods to override and place your strategy in:

`on_orderbook_update` - an asynchronous method called whenever an orderbook update is received from the Exchange socket

`on_portfolio_update` - an asynchronous method called whenever a user's portfolio updates on the Exchange

---

### Tools to Use

These are functions you should be calling to implement your strategy. They give you actions to interact with the exchange, methods to get information about the orderbook, and methods to view your stock portfolio.

* **Actions to Interact with Exchange**
  - `asyncio.create_task(self._quoter.place_limit(ticker=_, volume=_, price=_, is_bid=_))` - method to place limit order on exchange
  - `asyncio.create_task(self._quoter.place_market(ticker=_, volume=_, is_bid=_))` - method to place market order on exchange
  - `asyncio.create_task(self._quoter.remove_all())` - method to remove all open orders on exchange; use the web UI to remove individual order
* **Methods for Orderbook Information**
  - `self.get_orderbooks()` - returns orderbook's internal representation. looks like this:
  ```
  orderbook = {
            ticker1: {
                "bids": sortedcontainers.SortedDict({price1: volume1, price2: volume2,...}, reverse=True),
                "asks": sortedcontainers.SortedDict({{price1: volume1, price2: volume2,...}})
            },
            ticker2: {
                "bids": sortedcontainers.SortedDict({price1: volume1, price2: volume2,...}, reverse=True),
                "asks": sortedcontainers.SortedDict({{price1: volume1, price2: volume2,...}})
            },
            ...
        }
  ```

  - `self.best_bid(ticker=_)` - returns a float representing the best bid for the specified ticker
  - `self.best_ask(ticker=_)` - returns a float representing the best ask for the specified ticker
  - `self.mid(ticker=_)` - returns a float representing the midpoint for the specified ticker
  - `self.wmid(ticker=_)` - returns a float representing the weighted midpoint for the specified ticker
  - `self.spread(ticker=_)` - returns a float representing the spread of the specified ticker
* **Methods for Portfolio Information**
  - `self.get_positions()` - returns a representation of positions on each ticker. looks like this:
  ```
  {
    "ticker1": volume1,
    "ticker2": volume2,
    ...
  }
  ```
  - `self.get_orders()` - returns a representation of open orders. looks like this:
  ```
  {
    "ticker1": [Order1, Order2, ...],
    "ticker2": [Order1, Order2, ...],
    ...
  }
  ```
  - `self.get_balance()` - returns a float representing the balance
  - `self.get_pnl()` - returns a float reprenseting the estimated PnL

In [ ]:
class TestStrategy(Strategy):
    def __init__(self, quoter: Prioritizer, shared_state: SharedState):
        super().__init__(quoter, shared_state)
        self._cnt = 1

    async def on_orderbook_update(self) -> None:
        print("Orderbook update", self._cnt, time.time())
        # asyncio.create_task(self._quoter.remove_all())
        asyncio.create_task(self._quoter.place_limit(ticker="A", volume=1, price=50+self._cnt, is_bid=True))
        asyncio.create_task(self._quoter.place_limit(ticker="A", volume=1, price=950-self._cnt, is_bid=False))
        #asyncio.create_task(self._quoter.remove_all())
        # asyncio.create_task(self._quoter.place_market(ticker="A", volume=1, is_bid=True))
        # asyncio.create_task(self._quoter.place_market(ticker="A", volume=1, is_bid=False))
        self._cnt += 1

    async def on_portfolio_update(self) -> None:
        print("Portfolio update", self._cnt, time.time())
        print(self._shared_state.portfolio.positions)
        pass

## Main Loop

In [ ]:
async def start_strategy() -> None:
    """
    Async method to start a strategy.
    Returns: None

    """
    client = TradingClient(
        http_endpoint=URL,
        ws_endpoint=WS_URL,
        username=USERNAME,
        api_key=API_KEY,
    )
    shared_state = client.shared_state
    prioritizer = Prioritizer(rate_limit=RATE_LIMIT, trading_client=client)

    strategy: Strategy = TestStrategy(quoter=prioritizer, shared_state=shared_state)

    client.set_strategy(strategy=strategy)

    await strategy.start()

    await asyncio.sleep(1000000)


async def main() -> None:
    """
    Main async method for running all client tasks as asynchronous coroutines.
    Returns: None

    """
    tasks: list[asyncio.Task[None]] = [asyncio.create_task(start_strategy())]
    try:
        results = await asyncio.gather(
            *tasks,
            return_exceptions=True,
        )
        print(results)
    except Exception as e:
        print("Exception in main", e)
        traceback.print_exc()
        for task in tasks:
            task.cancel()
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    asyncio.run(main())